### Porting to Google Colab
The following cell enables this notebook to run from Google Colab as well as from your local machine IDE.<br>
You can change `root_directory` and/or `this_notebook_google_path` to point to the directory in your Google account, which contains this notebook, together with the `imgs` sub-directory and the rest of the files.<br>

In [1]:
import sys
import os
try:
    from google.colab import drive as google_drive # type: ignore
except:
    # no Google Colab --> fall back to local machine
    google_drive = None

if google_drive is not None:
    google_drive_directory = os.path.join('/','content','gdrive')
    google_drive.mount(google_drive_directory)
    all_projects_path = os.path.join(google_drive_directory, 'Othercomputers','My Laptop', 'projects')
else:
    all_projects_path = os.path.join('d:\\', 'projects')

project_path = os.path.join(all_projects_path,'RUNI','Thesis')
assert os.path.exists(project_path), f'Project path {project_path} not found!'
# enable import python files from this notebook's path
sys.path.append(project_path)
# enable reading images and data files from this notebook's path
os.chdir(project_path)

datasets_path = os.path.join(project_path, 'datasets')
assert os.path.exists(datasets_path), f'Datasets path {datasets_path} not found!'

output_path = os.path.join(project_path, 'output')
os.makedirs(output_path, exist_ok=True)
assert os.path.exists(output_path), f'Output path {output_path} not found!'

print(f'Current working directory: {os.getcwd()}')
print(f'Datasets path: {datasets_path}')
print(f'Output path: {output_path}')

Current working directory: d:\projects\RUNI\Thesis
Datasets path: d:\projects\RUNI\Thesis\datasets
Output path: d:\projects\RUNI\Thesis\output


In [2]:
from python.hpc import HybridArray


Numba version: 0.60.0
numba.njit is available.
CUDA is available and will be used for GPU operations.
Printing CUDA active device attributes:
    Name:                               NVIDIA GeForce GTX 1650
    Free Memory:                        3367680 [KB]
    Total Memory:                       4193984 [KB]
    Compute capability:                 7.5
    Clock rate:                         1560.00 [MHz]
    Memory clock rate:                  4001.00 [MHz]
    Memory bus width:                   128 bits
    Memory band width (theoretical)     128.03 [GByte/Sec]
    Number of multiprocessors:          16
    Minimal grid size:                  128
    Maximum grid size:                  (2147483647, 65535, 65535)
    Maximum block dimensions:           (1024, 1024, 64)
    Maximum threads per block:          1024
    Warp size:                          32
    Maximum shared memory per block:    49152 [bytes]
    Maximum registers per block:        65536
    Total constant memory:   

In [3]:

import numpy as np
from python.rare_weak_model.rare_weak_model import rare_weak_model
from python.adaptive_methods.adaptive_methods import apply_transform_discovery_method


Compiling Berk Jones for CUDA...
Compiling Berk Jones for CUDA... Done!


In [4]:
shape=(10,100)
epsilon = 0.1
mu = 1.0


In [5]:

def simulation(shape: tuple,\
               epsilon: float = 0.01, mu: float = 1.0,\
                **kwargs) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    use_gpu = kwargs.get('use_gpu', None)
    with (
        HybridArray() as sorted_p_values,
        HybridArray() as counts,
        HybridArray() as num_discoveries
    ):    
        sorted_p_values.realloc(shape=shape, dtype=np.float64, use_gpu=use_gpu)
        n1 = max(np.uint32(1),np.uint32(epsilon*shape[1]))
        rare_weak_model(sorted_p_values_output=sorted_p_values,\
                        cumulative_counts_output=counts,\
                        mu=mu, n1=n1, **kwargs)
        apply_transform_discovery_method(\
            sorted_p_values_input_output=sorted_p_values,\
            num_discoveries_output=num_discoveries,
            **kwargs)
        ret = (sorted_p_values.numpy(), counts.numpy(), num_discoveries.numpy())
    return ret

def simulation3(shape: tuple, epsilon: float = 0.01, mu: float = 1.0,\
                    **kwargs) -> None:
    print(f'Running on {kwargs=}')
    for gpu,njit in [(False,False), (False,True), (True,False)]:
        p_values, counts, num_discoveries = simulation(shape=shape, use_gpu=gpu, use_njit=njit,\
                                                       epsilon=epsilon, mu=mu,\
                                                        **kwargs)
        print(f'{gpu=} {njit=} --> p_values.mean={p_values.mean():.2e} counts.mean={counts.mean():.2f} num_discoveries.mean={num_discoveries.mean():.2f}')




In [6]:
simulation3(shape=shape, transform_method='higher_criticism', epsilon=epsilon, mu=mu)

Running on kwargs={'transform_method': 'higher_criticism'}
gpu=False njit=False --> p_values.mean=-6.32e-01 counts.mean=7.16 num_discoveries.mean=27.98
gpu=False njit=True --> p_values.mean=-6.32e-01 counts.mean=7.16 num_discoveries.mean=27.98


d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


gpu=True njit=False --> p_values.mean=-6.32e-01 counts.mean=7.16 num_discoveries.mean=27.98


d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [7]:
simulation3(shape=shape, transform_method='higher_criticism_unstable', epsilon=epsilon, mu=mu)

Running on kwargs={'transform_method': 'higher_criticism_unstable'}
gpu=False njit=False --> p_values.mean=-6.79e-01 counts.mean=7.16 num_discoveries.mean=11.47
gpu=False njit=True --> p_values.mean=-6.79e-01 counts.mean=7.16 num_discoveries.mean=11.47
gpu=True njit=False --> p_values.mean=-6.79e-01 counts.mean=7.16 num_discoveries.mean=11.47


d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [8]:
import math
from python.adaptive_methods.adaptive_methods import calc_lgamma

N = shape[1]
lgamma_gpu = HybridArray()
calc_lgamma(lgamma_gpu, N, use_gpu=True)
lgamma_njit = HybridArray()
calc_lgamma(lgamma_njit, N, use_gpu=False, use_njit=True)
lgamma_py = HybridArray()
calc_lgamma(lgamma_py, N, use_gpu=False, use_njit=False)
print(lgamma_gpu.numpy().sum())
print(lgamma_njit.numpy().sum())
print(lgamma_py.numpy().sum())


15980.934973073965
15980.934973073965
15980.934973073965


d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [9]:
simulation3(shape=shape, transform_method='berk_jones', epsilon=epsilon, mu=mu)

Running on kwargs={'transform_method': 'berk_jones'}
gpu=False njit=False --> p_values.mean=3.26e-01 counts.mean=7.16 num_discoveries.mean=18.91
gpu=False njit=True --> p_values.mean=3.26e-01 counts.mean=7.16 num_discoveries.mean=18.91
gpu=True njit=False --> p_values.mean=3.26e-01 counts.mean=7.16 num_discoveries.mean=18.91


d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
